In [33]:
import pandas as pd
import joblib
import json
from IPython.display import display, Markdown

def load_config(config_file="../config.json"):
    """Load configuration from a JSON file."""
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

In [34]:
test_df = pd.read_csv('../outputs/test_df.csv')

In [35]:
test_df

,projectid,total_price_excluding_optional_support,students_reached,total_projects_in_city,total_projects_in_state,percentage_reached_month_1,percentage_reached_month_2,percentage_reached_month_3,date_posted,fully_funded,...,month_posted_December,month_posted_February,month_posted_January,month_posted_July,month_posted_June,month_posted_March,month_posted_May,month_posted_November,month_posted_October,month_posted_September
0,a11d5c86692ef20e9f702094504bb5db,-0.011812,-0.026369,-0.604428,1.760114,-0.776143,-0.969951,-1.103760,2013-01-01,1.0,...,False,False,True,False,False,False,False,False,False,False
1,8bdc6b4cfc850f6f7cc3963c48ba454b,0.036787,0.008909,-0.619092,-1.032311,-0.594878,-0.798105,-0.932964,2013-01-01,0.0,...,False,False,True,False,False,False,False,False,False,False
2,88f263f1cd657bccce254ffc0b79df4d,0.023713,-0.028831,-0.627791,1.760114,1.452347,1.142747,0.996033,2013-01-01,1.0,...,False,False,True,False,False,False,False,False,False,False
3,a8a6e08b6ae7ab1a88c16ba82ac8e9da,-0.002366,-0.000936,-0.614867,-0.745992,1.452347,1.142747,0.996033,2013-01-01,1.0,...,False,False,True,False,False,False,False,False,False,False
4,be21151602e4fd47dd4012114fd519c6,-0.022787,-0.030061,-0.598588,-0.661352,-0.379132,1.142747,0.996033,2013-01-01,1.0,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114032,a0e839f24645e3d6dcbd327f8441b043,0.001489,-0.015704,1.890146,0.566334,0.411419,1.142747,0.996033,2013-12-31,1.0,...,True,False,False,False,False,False,False,False,False,False
114033,f820ef3537f4445b0716244fae36f763,-0.019920,-0.028010,-0.633631,-1.034493,-0.776143,-0.969951,-1.103760,2013-12-31,0.0,...,True,False,False,False,False,False,False,False,False,False
114034,95ee208a51831edffa7cc2e0aa3e83cd,-0.025463,-0.000116,-0.274494,-0.858846,1.452347,1.142747,0.996033,2013-12-31,1.0,...,True,False,False,False,False,False,False,False,False,False
114035,1aa5e1d739a40c2763da1a0bb0f0f335,-0.027321,-0.030882,-0.534216,-0.690961,1.452347,1.142747,0.996033,2013-12-31,1.0,...,True,False,False,False,False,False,False,False,False,False


In [36]:
test_df.columns

Index(['projectid', 'total_price_excluding_optional_support',
       'students_reached', 'total_projects_in_city', 'total_projects_in_state',
       'percentage_reached_month_1', 'percentage_reached_month_2',
       'percentage_reached_month_3', 'date_posted', 'fully_funded',
       'poverty_level_high poverty', 'poverty_level_low poverty',
       'school_metro_rural', 'school_metro_suburban', 'school_metro_urban',
       'grade_level_Grades 3-5', 'grade_level_Grades 6-8',
       'grade_level_Grades 9-12', 'grade_level_Grades PreK-2',
       'primary_focus_subject_Applied Sciences',
       'primary_focus_subject_Character Education',
       'primary_focus_subject_Civics & Government',
       'primary_focus_subject_College & Career Prep',
       'primary_focus_subject_Community Service', 'primary_focus_subject_ESL',
       'primary_focus_subject_Early Development',
       'primary_focus_subject_Economics',
       'primary_focus_subject_Environmental Science',
       'primary_focus_subje

In [37]:
test_df['date_posted'] = pd.to_datetime(test_df['date_posted'])

In [38]:
# Set max_date to filter projects posted for at least 3 months
fixed_max_date = pd.to_datetime('2013-12-31')
test_df['months_since_posted'] = ((fixed_max_date - test_df['date_posted']) / pd.Timedelta(days=30)).astype(int)
eligible_projects = test_df[test_df['months_since_posted'] >= 3].copy()

In [39]:
test_df.columns

Index(['projectid', 'total_price_excluding_optional_support',
       'students_reached', 'total_projects_in_city', 'total_projects_in_state',
       'percentage_reached_month_1', 'percentage_reached_month_2',
       'percentage_reached_month_3', 'date_posted', 'fully_funded',
       'poverty_level_high poverty', 'poverty_level_low poverty',
       'school_metro_rural', 'school_metro_suburban', 'school_metro_urban',
       'grade_level_Grades 3-5', 'grade_level_Grades 6-8',
       'grade_level_Grades 9-12', 'grade_level_Grades PreK-2',
       'primary_focus_subject_Applied Sciences',
       'primary_focus_subject_Character Education',
       'primary_focus_subject_Civics & Government',
       'primary_focus_subject_College & Career Prep',
       'primary_focus_subject_Community Service', 'primary_focus_subject_ESL',
       'primary_focus_subject_Early Development',
       'primary_focus_subject_Economics',
       'primary_focus_subject_Environmental Science',
       'primary_focus_subje

In [40]:
config = load_config()
poverty_levels = ["low", "high"]
models = ["random_forest", "logistic_regression"]
recommendations = {}

In [41]:
for model_type in models:
    for pov_level in poverty_levels:
        pov_column = f"poverty_level_{pov_level} poverty"
        pov_projects = eligible_projects[eligible_projects[pov_column] == 1].copy()
        
        # Load the classifier
        classifier = joblib.load(f"../outputs/{model_type}_{pov_level}_poverty.pkl")
        
        X_test_filtered = pov_projects.drop(columns=['fully_funded', 'date_posted', 'months_since_posted', 'projectid'], errors='ignore')
        X_test_filtered = X_test_filtered.reindex(columns=classifier.feature_names_in_, fill_value=0)
        
        pov_projects[f'probability_fully_funded_{model_type}'] = classifier.predict_proba(X_test_filtered)[:, 1]
        
        pov_projects_sorted = pov_projects.sort_values(by=f'probability_fully_funded_{model_type}', ascending=False)

        if pov_level in ["high", "highest"]:
            top_recommendations = pov_projects_sorted.head(10)
        elif pov_level in ["low", "moderate"]:
            top_recommendations = pov_projects_sorted.head(3)

        recommendations[(model_type, pov_level)] = top_recommendations[['projectid', 'date_posted', f'probability_fully_funded_{model_type}']]

In [42]:
list(recommendations.keys())

[('random_forest', 'low'),
 ('random_forest', 'high'),
 ('logistic_regression', 'low'),
 ('logistic_regression', 'high')]

In [43]:
for key, recs in recommendations.items():
    model_type, pov_level = key  # Unpack model type and poverty level
    
    # Sort projects by probability to create a ranking
    recs_sorted = recs.sort_values(f'probability_fully_funded_{model_type}', ascending=False).reset_index(drop=True)
    recs_sorted['rank'] = recs_sorted.index + 1  # Add rank column
    
    # Add model type and poverty level columns
    recs_sorted['model_type'] = model_type.capitalize()
    recs_sorted['poverty_level'] = pov_level.capitalize()
    
    # Select only relevant columns and rename for consistency
    table = recs_sorted[['rank', 'projectid', 'model_type', 'poverty_level', f'probability_fully_funded_{model_type}']]
    table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)
    
    # Display table with a header for context
    display(Markdown(f"### {model_type.capitalize()} Model - {pov_level.capitalize()} Poverty Level"))
    display(table)

/var/folders/5m/4tj5fx915fv1z073c22c2q7m0000gn/T/ipykernel_83789/692742374.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)


### Random_forest Model - Low Poverty Level

,rank,projectid,model_type,poverty_level,probability_fully_funded
0,1,e83030a9ec326255c77fcc9cdb663e39,Random_forest,Low,1.0
1,2,1b00ed7b744ee58b1473d810a703a71f,Random_forest,Low,1.0
2,3,65eedd892fb5365449a06a3ca4d87a0a,Random_forest,Low,1.0


/var/folders/5m/4tj5fx915fv1z073c22c2q7m0000gn/T/ipykernel_83789/692742374.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)


### Random_forest Model - High Poverty Level

...

### Logistic_regression Model - Low Poverty Level

,rank,projectid,model_type,poverty_level,probability_fully_funded
0,1,ec359f64bf3ec063c2b3442a4c2ee52c,Logistic_regression,Low,1.0
1,2,cb09d5c2747675fd1a0d9256665634ae,Logistic_regression,Low,1.0
2,3,b108a17ac554632bc10e9f10a0032bc0,Logistic_regression,Low,1.0


/var/folders/5m/4tj5fx915fv1z073c22c2q7m0000gn/T/ipykernel_83789/692742374.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.rename(columns={f'probability_fully_funded_{model_type}': 'probability_fully_funded'}, inplace=True)


### Logistic_regression Model - High Poverty Level

,rank,projectid,model_type,poverty_level,probability_fully_funded
0,1,23093b3e4e0749f365c5637fd19258fa,Logistic_regression,High,1.0
1,2,e891154767af72f5889a4ee9c269ccfc,Logistic_regression,High,1.0
2,3,c2be8cea38597dccac6cc66f61cbe219,Logistic_regression,High,1.0
3,4,b98d990d3f8a8059852296f34bc6754f,Logistic_regression,High,1.0
4,5,2c755e2c7b3ac84a83e12c05b0c45cb2,Logistic_regression,High,1.0
5,6,8bcfd367af337e9088f5b0de720ca36b,Logistic_regression,High,1.0
6,7,aadfbc4556e1696ba544f912ad90bbc7,Logistic_regression,High,1.0
7,8,38f83be57ec9d2c3909411d75eeec2b4,Logistic_regression,High,1.0
8,9,7b65f46e12488e12244c821a9283f2e0,Logistic_regression,High,1.0
9,10,0bd1420971b22fc70057e5462dc9f093,Logistic_regression,High,1.0


In [44]:
for (model_type, pov_level), top_recs in recommendations.items():
    top_recs = top_recs.merge(test_df, on='projectid', how='left', suffixes=('', '_test'))
    recommendations[(model_type, pov_level)] = top_recs
    display(recommendations)

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e83030a9ec326255c77fcc9cdb663e39  2013-07-10   
 1  1b00ed7b744ee58b1473d810a703a71f  2013-08-17   
 2  65eedd892fb5365449a06a3ca4d87a0a  2013-08-17   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  \
 0                               -0.015421         -0.028831   
 1                                0.008988         -0.030471   
 2                                0.026836         -0.031292   
 
    total_projects_in_city  total_projects_in_state  \
 0               -0.624933                -0.978494   
 1               -0.544779                -0.176461   
 2               -0.487118                -0.745992   
 
    percentage_reached_month_1  percentage_reached_month_2  \
 0                    0.113224    

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e83030a9ec326255c77fcc9cdb663e39  2013-07-10   
 1  1b00ed7b744ee58b1473d810a703a71f  2013-08-17   
 2  65eedd892fb5365449a06a3ca4d87a0a  2013-08-17   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  \
 0                               -0.015421         -0.028831   
 1                                0.008988         -0.030471   
 2                                0.026836         -0.031292   
 
    total_projects_in_city  total_projects_in_state  \
 0               -0.624933                -0.978494   
 1               -0.544779                -0.176461   
 2               -0.487118                -0.745992   
 
    percentage_reached_month_1  percentage_reached_month_2  \
 0                    0.113224    

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e83030a9ec326255c77fcc9cdb663e39  2013-07-10   
 1  1b00ed7b744ee58b1473d810a703a71f  2013-08-17   
 2  65eedd892fb5365449a06a3ca4d87a0a  2013-08-17   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  \
 0                               -0.015421         -0.028831   
 1                                0.008988         -0.030471   
 2                                0.026836         -0.031292   
 
    total_projects_in_city  total_projects_in_state  \
 0               -0.624933                -0.978494   
 1               -0.544779                -0.176461   
 2               -0.487118                -0.745992   
 
    percentage_reached_month_1  percentage_reached_month_2  \
 0                    0.113224    

{('random_forest',
  'low'):                           projectid date_posted  \
 0  e83030a9ec326255c77fcc9cdb663e39  2013-07-10   
 1  1b00ed7b744ee58b1473d810a703a71f  2013-08-17   
 2  65eedd892fb5365449a06a3ca4d87a0a  2013-08-17   
 
    probability_fully_funded_random_forest  \
 0                                     1.0   
 1                                     1.0   
 2                                     1.0   
 
    total_price_excluding_optional_support  students_reached  \
 0                               -0.015421         -0.028831   
 1                                0.008988         -0.030471   
 2                                0.026836         -0.031292   
 
    total_projects_in_city  total_projects_in_state  \
 0               -0.624933                -0.978494   
 1               -0.544779                -0.176461   
 2               -0.487118                -0.745992   
 
    percentage_reached_month_1  percentage_reached_month_2  \
 0                    0.113224    

In [14]:
stem_subjects = ['primary_focus_subject_Applied Sciences',
                 'primary_focus_subject_Health & Life Science',
                 'primary_focus_subject_Environmental Science',
                 'primary_focus_subject_Mathematics']

def classify_stem(row):
    # Check if any STEM subject column is 1
    return 'STEM' if row[stem_subjects].sum() > 0 else 'Non-STEM'

test_df['STEM_category'] = test_df.apply(classify_stem, axis=1)

In [15]:
for (model_type, pov_level), df in recommendations.items():
    df = df.drop_duplicates().reset_index(drop=True)  # Drop any existing duplicates
    df = df.reset_index(drop=True)
    recommendations[(model_type, pov_level)] = df.assign(model_type=model_type, pov_level=pov_level)

# Concatenate all DataFrames from recommendations
all_recs = pd.concat(recommendations.values(), ignore_index=True)

In [17]:
all_recs.columns

Index(['projectid', 'date_posted', 'probability_fully_funded_random_forest',
       'total_price_excluding_optional_support', 'students_reached',
       'fully_funded', 'date_posted_test', 'total_projects_in_city',
       'total_projects_in_state', 'percentage_reached_month_1',
       'percentage_reached_month_2', 'percentage_reached_month_3',
       'poverty_level_high poverty', 'poverty_level_highest poverty',
       'poverty_level_low poverty', 'poverty_level_moderate poverty',
       'school_metro_rural', 'school_metro_suburban', 'school_metro_urban',
       'grade_level_Grades 3-5', 'grade_level_Grades 6-8',
       'grade_level_Grades 9-12', 'grade_level_Grades PreK-2',
       'primary_focus_subject_Applied Sciences',
       'primary_focus_subject_Character Education',
       'primary_focus_subject_Civics & Government',
       'primary_focus_subject_College & Career Prep',
       'primary_focus_subject_Community Service', 'primary_focus_subject_ESL',
       'primary_focus_subject_

In [18]:
all_recs

,projectid,date_posted,probability_fully_funded_random_forest,total_price_excluding_optional_support,students_reached,fully_funded,date_posted_test,total_projects_in_city,total_projects_in_state,percentage_reached_month_1,...,month_posted_June,month_posted_March,month_posted_May,month_posted_November,month_posted_October,month_posted_September,months_since_posted,model_type,pov_level,probability_fully_funded_logistic_regression
0,e94a5f08bb09a5c745edc4860b0d85ac,2013-01-01,1.0,752.10,30.0,1.0,2013-01-01,392,126242,100.000000,...,False,False,False,False,False,False,12,random_forest,low,NaN
1,d97caf8b7243ba9128cc0a7aacdf9780,2013-08-30,1.0,255.22,160.0,1.0,2013-08-30,131,126242,100.000000,...,False,False,False,False,False,False,4,random_forest,low,NaN
2,efd84a1f24856df0431a469a3abf4664,2013-08-25,1.0,450.50,23.0,1.0,2013-08-25,114,7021,100.000000,...,False,False,False,False,False,False,4,random_forest,low,NaN
3,0c61d6de55e1ec14bcf76540c132fee7,2013-10-02,1.0,431.14,23.0,1.0,2013-10-02,1965,10716,0.000000,...,False,False,False,False,True,False,3,random_forest,moderate,NaN
4,b313d50d7e4448569f988061630cb539,2013-04-14,1.0,378.53,35.0,1.0,2013-04-14,357,126242,100.000000,...,False,False,False,False,False,False,8,random_forest,moderate,NaN
5,b7bf7820c5110ee7db3949b51a5a9c7d,2013-08-29,1.0,426.35,160.0,1.0,2013-08-29,11,126242,100.000000,...,False,False,False,False,False,False,4,random_forest,moderate,NaN
6,88f263f1cd657bccce254ffc0b79df4d,2013-01-01,1.0,868.02,28.0,1.0,2013-01-01,102,126242,100.000000,...,False,False,False,False,False,False,12,random_forest,high,NaN
7,4ebd0837defce812ff9d312c3f78db33,2013-08-04,1.0,542.62,25.0,1.0,2013-08-04,1428,43478,100.000000,...,False,False,False,False,False,False,4,random_forest,high,NaN
8,3fe2b20e3ab2c4f00d20a217329f2dc3,2013-08-04,1.0,156.05,25.0,1.0,2013-08-04,943,10469,100.000000,...,False,False,False,False,False,False,4,random_forest,high,NaN
9,cc861ce787eb2377f5525755c8d2f159,2013-08-04,1.0,224.29,23.0,1.0,2013-08-04,98,18615,0.000000,...,False,False,False,False,False,False,4,random_forest,high,NaN
